In [26]:
import pandas as pd
import numpy as np

# Load the data
def return_cont(player, path):
    # deuce_scores = ["0-0", "30-0", "0-30", "15-15", "40-40"]
    # & (events['pointScore'].isin(deuce_scores))
    events = pd.read_csv(path)
    events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()
    returns_ucla = events[(events['shotHitBy'] == player) & (events['shotInRally'] == 2)][['pointStartTime', 'shotHitBy', 'shotContactX', 'shotContactY', 'pointWonBy', 'isWinner', 'shotFhBh']].dropna(subset=['pointWonBy', 'shotContactX', 'shotContactY']).copy()
    returns_ucla['shotContactX'] = returns_ucla.apply(lambda row: -row['shotContactX'] if row['shotContactY'] > 0 else row['shotContactX'], axis=1)
    returns_ucla['shotContactY'] = returns_ucla['shotContactY'].apply(lambda y: -y if y > 0 else y)
    returns_ucla['depth'] = returns_ucla['shotContactY'].apply(
        lambda y: 'short' if y >= -455 else 'mid' if -455 > y > -490 else 'deep'
    )

    # Calculate count + win pct.
    distribution = returns_ucla.groupby('depth').apply(
        lambda df: pd.Series({
            'freq': len(df),
            'win_percentage': int((df['pointWonBy'] == df['shotHitBy']).mean() * 100)
        })
    ).reset_index()

    max_win_percentage = distribution['win_percentage'].max()
    min_win_percentage = distribution['win_percentage'].min()

    distribution['maxMin'] = distribution['win_percentage'].apply(
        lambda x: 'max' if x == max_win_percentage else 'min' if x == min_win_percentage else 'no'
    )

    distribution['win_percentage'] = distribution['win_percentage'].astype(str) + '%'

    y_mapping = {
        'short': {'y': -420},
        'mid': {'y': -475},
        'deep': {'y': -515}
    }

    distribution['y'] = distribution['depth'].map(lambda d: y_mapping[d]['y'])

    # Export the data
    returns_ucla_json = returns_ucla.to_json(orient='records')
    distribution_json = distribution.to_json(orient='records')

    with open('returns_ucla.json', 'w') as f:
        f.write(returns_ucla_json)

    with open('distribution.json', 'w') as f:
        f.write(distribution_json)

player = "Anne Lutkemeyer"
path = "../Match CSVs/Shot_Visuals_AnneLutkemeyer_CatherineGagnon.csv"
return_cont(player, path)

/var/folders/gc/y_rdwxcs2pb0n4dz4z33dcth0000gn/T/ipykernel_6417/1267620436.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  distribution = returns_ucla.groupby('depth').apply(
